# Initial conditions:

In [31]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

In [32]:
api.dataset_download_file(dataset="teertha/ushealthinsurancedataset", file_name="insurance.csv")
main_data = pd.read_csv("insurance.csv")


Dataset URL: https://www.kaggle.com/datasets/teertha/ushealthinsurancedataset


In [33]:
main_data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


# Cleaning the data:

In [34]:
main_data = main_data.round(decimals=2)

In [35]:
main_data.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

In [36]:
print(main_data.duplicated().sum())
main_data[main_data.duplicated()]

1


,age,sex,bmi,children,smoker,region,charges
581,19,male,30.59,0,no,northwest,1639.56


In [37]:
main_data.drop_duplicates(inplace = True)
print(main_data.duplicated().sum())

0


# Pre-analysis data hypotheses:

- older people would pay a higher premium
- people with a higher bmi would pay a higher premium
- region with the highest average charges would be the most wealthy on average

# Cursory view of data trends:

In [44]:
main_data.describe()

,age,bmi,children,charges
count,1337.000000,1337.000000,1337.000000,1337.000000
mean,39.222139,30.663628,1.095737,13279.121503
std,14.044333,6.100233,1.205571,12110.359677
min,18.000000,15.960000,0.000000,1121.870000
25%,27.000000,26.290000,0.000000,4746.340000
50%,39.000000,30.400000,1.000000,9386.160000
75%,51.000000,34.700000,2.000000,16657.720000
max,64.000000,53.130000,5.000000,63770.430000


In [45]:
main_data.describe(include="object")

,sex,smoker,region
count,1337,1337,1337
unique,2,2,4
top,male,no,southeast
freq,675,1063,364


In [53]:
print(main_data["sex"].value_counts())
print(main_data["smoker"].value_counts())
print(main_data["region"].value_counts())

sex
male      675
female    662
Name: count, dtype: int64
smoker
no     1063
yes     274
Name: count, dtype: int64
region
southeast    364
southwest    325
northwest    324
northeast    324
Name: count, dtype: int64


most and least expensive insurance charges:

In [38]:
main_data.sort_values(by='charges', ascending=False).head(10)

,age,sex,bmi,children,smoker,region,charges
543,54,female,47.41,0,yes,southeast,63770.43
1300,45,male,30.36,0,yes,southeast,62592.87
1230,52,male,34.48,3,yes,northwest,60021.40
577,31,female,38.10,1,yes,northeast,58571.07
819,33,female,35.53,0,yes,northwest,55135.40
1146,60,male,32.80,0,yes,southwest,52590.83
34,28,male,36.40,1,yes,southwest,51194.56
1241,64,male,36.96,2,yes,southeast,49577.66
1062,59,male,41.14,1,yes,southeast,48970.25
488,44,female,38.06,0,yes,southeast,48885.14


In [39]:
main_data.sort_values(by='charges', ascending=False).tail(10)

,age,sex,bmi,children,smoker,region,charges
1317,18,male,53.13,0,no,southeast,1163.46
442,18,male,43.01,0,no,southeast,1149.40
781,18,male,41.14,0,no,southeast,1146.80
866,18,male,37.29,0,no,southeast,1141.45
194,18,male,34.43,0,no,southeast,1137.47
22,18,male,34.10,0,no,southeast,1137.01
663,18,male,33.66,0,no,southeast,1136.40
1244,18,male,33.33,0,no,southeast,1135.94
808,18,male,30.14,0,no,southeast,1131.51
940,18,male,23.21,0,no,southeast,1121.87


highest and lowest BMIs:

In [40]:
main_data.sort_values(by='bmi', ascending=False).head(10)

,age,sex,bmi,children,smoker,region,charges
1317,18,male,53.13,0,no,southeast,1163.46
1047,22,male,52.58,1,yes,southeast,44501.40
847,23,male,50.38,1,no,southeast,2438.06
116,58,male,49.06,0,no,southeast,11381.33
286,46,female,48.07,2,no,northeast,9432.93
1088,52,male,47.74,1,no,southeast,9748.91
860,37,female,47.60,2,yes,southwest,46113.51
401,47,male,47.52,1,no,southeast,8083.92
543,54,female,47.41,0,yes,southeast,63770.43
438,52,female,46.75,5,no,southeast,12592.53


In [43]:
main_data.sort_values(by='bmi', ascending=False).tail(10)

,age,sex,bmi,children,smoker,region,charges
410,19,male,17.48,0,no,northwest,1621.34
680,21,female,17.40,1,no,southwest,2585.27
28,23,male,17.39,1,no,northwest,2775.19
1029,37,female,17.29,2,no,northeast,6877.98
250,18,male,17.29,2,yes,northeast,12829.46
1286,28,female,17.29,0,no,northeast,3732.63
412,26,female,17.20,2,yes,northeast,14455.64
428,21,female,16.82,1,no,northeast,3167.46
1226,38,male,16.82,2,no,northeast,6640.54
172,18,male,15.96,0,no,northeast,1694.80


# Groupby analysis:

In [108]:
main_data.groupby("sex")["charges"].mean().round(2)

sex
female    12569.58
male      13975.00
Name: charges, dtype: float64

In [107]:
main_data.groupby('region')['charges'].agg(["mean", "median"]).round(2)

,mean,median
region,,
northeast,13406.38,10057.65
northwest,12450.84,8976.98
southeast,14735.41,9294.13
southwest,12346.94,8798.59


In [106]:
main_data['age_group'] = pd.cut(main_data['age'], bins=[17, 30, 40, 50, 60, 70], labels=['18-29', '30-39', '40-49', '50-59', '60-69'])
main_data.groupby('age_group')['charges'].mean().round(2)

/var/folders/7m/3h9k282d7dqfggyrf3bb01lc0000gp/T/ipykernel_33040/3829735877.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  main_data.groupby('age_group')['charges'].mean().round(2)


age_group
18-29     9415.06
30-39    11639.31
40-49    14782.04
50-59    17062.29
60-69    21063.16
Name: charges, dtype: float64

In [75]:
main_data.groupby("smoker")["charges"].mean()

smoker
no      8440.660310
yes    32050.231898
Name: charges, dtype: float64

In [105]:
main_data["bmi_group"] = pd.cut(main_data["bmi"], bins=[15.5, 18.5, 24.9, 29.9, 34.9, 39.9, 53.13] , labels=['Underweight (<18.5)', 'Normal (18.5-24.9)', 'Overweight (25-29.9)', 'Obese Class I (30-34.9)', 'Obese Class II (35-39.9)', 'Obese Class III (>=40)']
)
main_data.groupby("bmi_group")["charges"].mean().round(2)

/var/folders/7m/3h9k282d7dqfggyrf3bb01lc0000gp/T/ipykernel_33040/1638727608.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  main_data.groupby("bmi_group")["charges"].mean().round(2)


bmi_group
Underweight (<18.5)          8657.62
Normal (18.5-24.9)          10404.90
Overweight (25-29.9)        11006.81
Obese Class I (30-34.9)     14249.38
Obese Class II (35-39.9)    17245.41
Obese Class III (>=40)      16667.61
Name: charges, dtype: float64

In [104]:
main_data.groupby(["bmi_group", "smoker"])["charges"].mean().round(2)

/var/folders/7m/3h9k282d7dqfggyrf3bb01lc0000gp/T/ipykernel_33040/3325694726.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  main_data.groupby(["bmi_group", "smoker"])["charges"].mean().round(2)


bmi_group                 smoker
Underweight (<18.5)       no         5485.06
                          yes       18809.83
Normal (18.5-24.9)        no         7616.21
                          yes       19942.22
Overweight (25-29.9)      no         8348.37
                          yes       22379.03
Obese Class I (30-34.9)   no         8514.35
                          yes       39204.50
Obese Class II (35-39.9)  no         9621.30
                          yes       42756.86
Obese Class III (>=40)    no         8267.56
                          yes       45467.79
Name: charges, dtype: float64

# Importing second database:

In [76]:
html = """
<tbody><tr><th>Rank</th><th>State</th><th>FIPS Code</th><th>PCPI ($)</th></tr><tr><td>1</td><td class="text-left">District of Columbia</td><td>11000</td><td>100,909</td></tr><tr><td>2</td><td class="text-left">Massachusetts</td><td>25000</td><td>87,812</td></tr><tr><td>3</td><td class="text-left">Connecticut</td><td>09000</td><td>87,447</td></tr><tr><td>4</td><td class="text-left">New Jersey</td><td>34000</td><td>80,724</td></tr><tr><td>5</td><td class="text-left">California</td><td>06000</td><td>80,423</td></tr><tr><td>6</td><td class="text-left">Washington</td><td>53000</td><td>79,659</td></tr><tr><td>7</td><td class="text-left">New York</td><td>36000</td><td>79,581</td></tr><tr><td>8</td><td class="text-left">Colorado</td><td>08000</td><td>78,918</td></tr><tr><td>9</td><td class="text-left">Wyoming</td><td>56000</td><td>77,837</td></tr><tr><td>10</td><td class="text-left">New Hampshire</td><td>33000</td><td>77,260</td></tr><tr><td>11</td><td class="text-left">Maryland</td><td>24000</td><td>73,849</td></tr><tr><td>12</td><td class="text-left">North Dakota</td><td>38000</td><td>73,341</td></tr><tr><td>13</td><td class="text-left">Virginia</td><td>51000</td><td>72,855</td></tr><tr><td>14</td><td class="text-left">Minnesota</td><td>27000</td><td>71,866</td></tr><tr><td>15</td><td class="text-left">Alaska</td><td>02000</td><td>71,616</td></tr><tr><td>16</td><td class="text-left">Illinois</td><td>17000</td><td>70,953</td></tr><tr><td>17</td><td class="text-left">South Dakota</td><td>46000</td><td>70,353</td></tr><tr><td>18</td><td class="text-left">Florida</td><td>12000</td><td>68,248</td></tr><tr><td>19</td><td class="text-left">Pennsylvania</td><td>42000</td><td>67,839</td></tr><tr><td>20</td><td class="text-left">Nebraska</td><td>31000</td><td>67,800</td></tr><tr><td>21</td><td class="text-left">Rhode Island</td><td>44000</td><td>66,480</td></tr><tr><td>22</td><td class="text-left">Vermont</td><td>50000</td><td>66,463</td></tr><tr><td>23</td><td class="text-left">Oregon</td><td>41000</td><td>65,426</td></tr><tr><td>24</td><td class="text-left">Texas</td><td>48000</td><td>65,422</td></tr><tr><td>25</td><td class="text-left">Delaware</td><td>10000</td><td>65,392</td></tr><tr><td>26</td><td class="text-left">Nevada</td><td>32000</td><td>65,168</td></tr><tr><td>27</td><td class="text-left">Hawaii</td><td>15000</td><td>65,151</td></tr><tr><td>28</td><td class="text-left">Wisconsin</td><td>55000</td><td>63,963</td></tr><tr><td>29</td><td class="text-left">Montana</td><td>30000</td><td>63,918</td></tr><tr><td>30</td><td class="text-left">Kansas</td><td>20000</td><td>63,732</td></tr><tr><td>31</td><td class="text-left">Maine</td><td>23000</td><td>63,117</td></tr><tr><td>32</td><td class="text-left">Utah</td><td>49000</td><td>62,823</td></tr><tr><td>33</td><td class="text-left">Iowa</td><td>19000</td><td>62,351</td></tr><tr><td>34</td><td class="text-left">Arizona</td><td>04000</td><td>61,652</td></tr><tr><td>35</td><td class="text-left">Missouri</td><td>29000</td><td>61,302</td></tr><tr><td>36</td><td class="text-left">Tennessee</td><td>47000</td><td>61,049</td></tr><tr><td>37</td><td class="text-left">North Carolina</td><td>37000</td><td>60,484</td></tr><tr><td>38</td><td class="text-left">Ohio</td><td>39000</td><td>60,402</td></tr><tr><td>39</td><td class="text-left">Indiana</td><td>18000</td><td>60,038</td></tr><tr><td>40</td><td class="text-left">Michigan</td><td>26000</td><td>59,714</td></tr><tr><td>41</td><td class="text-left">Idaho</td><td>16000</td><td>59,035</td></tr><tr><td>42</td><td class="text-left">Georgia</td><td>13000</td><td>58,581</td></tr><tr><td>43</td><td class="text-left">Oklahoma</td><td>40000</td><td>58,499</td></tr><tr><td>44</td><td class="text-left">Louisiana</td><td>22000</td><td>57,100</td></tr><tr><td>45</td><td class="text-left">South Carolina</td><td>45000</td><td>56,123</td></tr><tr><td>46</td><td class="text-left">New Mexico</td><td>35000</td><td>54,428</td></tr><tr><td>47</td><td class="text-left">Arkansas</td><td>05000</td><td>54,347</td></tr><tr><td>48</td><td class="text-left">Kentucky</td><td>21000</td><td>54,326</td></tr><tr><td>49</td><td class="text-left">Alabama</td><td>01000</td><td>53,175</td></tr><tr><td>50</td><td class="text-left">West Virginia</td><td>54000</td><td>52,585</td></tr><tr><td>51</td><td class="text-left">Mississippi</td><td>28000</td><td>48,110</td></tr></tbody>
"""

soup = BeautifulSoup(html, 'html.parser')

rows = soup.find_all('tr')[1:]

data = []
for row in rows:
    row_data = [cell.get_text(strip=True) for cell in row.find_all('td')]
    data.append(row_data)

states_pcpi = pd.DataFrame(data, columns=['Rank', 'State', 'FIPS Code', 'PCPI ($)'])

states_pcpi

,Rank,State,FIPS Code,PCPI ($)
0,1,District of Columbia,11000,"100,909"
1,2,Massachusetts,25000,"87,812"
2,3,Connecticut,09000,"87,447"
3,4,New Jersey,34000,"80,724"
4,5,California,06000,"80,423"
5,6,Washington,53000,"79,659"
6,7,New York,36000,"79,581"
7,8,Colorado,08000,"78,918"
8,9,Wyoming,56000,"77,837"
9,10,New Hampshire,33000,"77,260"


Mapping the states to regions to be consistent with the original dataframe

In [91]:
us_state_regions_dict = {
    "Alabama": "southeast",
    "Alaska": "northwest",
    "Arizona": "southwest",
    "Arkansas": "southeast",
    "California": "southwest",
    "Colorado": "southwest",
    "Connecticut": "northeast",
    "Delaware": "northeast",
    "Florida": "southeast",
    "Georgia": "southeast",
    "Hawaii": "southeast",
    "Idaho": "northwest",
    "Illinois": "northeast",
    "Indiana": "northeast",
    "Iowa": "northeast",
    "Kansas": "southwest",
    "Kentucky": "southeast",
    "Louisiana": "southeast",
    "Maine": "northeast",
    "Maryland": "southeast",
    "Massachusetts": "northeast",
    "Michigan": "northeast",
    "Minnesota": "northeast",
    "Mississippi": "southeast",
    "Missouri": "southeast",
    "Montana": "northwest",
    "Nebraska": "northwest",
    "Nevada": "southwest",
    "New Hampshire": "northeast",
    "New Jersey": "northeast",
    "New Mexico": "southwest",
    "New York": "northeast",
    "North Carolina": "southeast",
    "North Dakota": "northwest",
    "Ohio": "northeast",
    "Oklahoma": "southwest",
    "Oregon": "northwest",
    "Pennsylvania": "northeast",
    "Rhode Island": "northeast",
    "South Carolina": "southeast",
    "South Dakota": "northwest",
    "Tennessee": "southeast",
    "Texas": "southwest",
    "Utah": "southwest",
    "Vermont": "northeast",
    "Virginia": "southeast",
    "Washington": "northwest",
    "West Virginia": "southeast",
    "Wisconsin": "northeast",
    "Wyoming": "northwest"
}
states_pcpi['Region'] = states_pcpi['State'].map(us_state_regions_dict)
states_pcpi.drop(columns=["FIPS Code"], inplace = True)
states_pcpi.fillna("northeast", inplace = True)
states_pcpi.set_index('Rank', inplace=True)

In [92]:
states_pcpi['PCPI ($)'] = states_pcpi['PCPI ($)'].str.replace(",","").astype("int64")

In [93]:
states_pcpi

,State,PCPI ($),Region
Rank,,,
1,District of Columbia,100909,northeast
2,Massachusetts,87812,northeast
3,Connecticut,87447,northeast
4,New Jersey,80724,northeast
5,California,80423,southwest
6,Washington,79659,northwest
7,New York,79581,northeast
8,Colorado,78918,southwest
9,Wyoming,77837,northwest


# Personal Income pivot table

In [103]:
states_pcpi.pivot_table(index='Region', values=['PCPI ($)'], aggfunc='mean').round(2)

,PCPI ($)
Region,
northeast,71795.06
northwest,69887.22
southeast,59819.00
southwest,65673.89


# Importing third dataframe

In [110]:
cancer_url = "https://www.statecancerprofiles.cancer.gov/incidencerates/index.php?stateFIPS=00&areatype=state&cancer=001&race=00&sex=0&age=001&year=0&type=incd&sortVariableName=rate&sortOrder=default&output=0#results"
cancer_response = requests.get(cancer_url)
cancer_soup = BeautifulSoup(cancer_response.content, "html.parser")

In [111]:
cancer_soup


<!DOCTYPE HTML>

<html lang="en">
<head>
<!-- Global site tag (gtag.js) - Google Analytics -->
<script async="" src="https://www.googletagmanager.com/gtag/js?id=UA-112281461-1"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());
  gtag('config', 'UA-112281461-1');
</script>
<!-- Regular Header -->
<title>State Cancer Profiles &gt; Incidence Rates Table</title>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport">
<meta description="">
<link href="/bootstrap/css/bootstrap.min.css" rel="stylesheet"/>
<link href="/favicon.ico" rel="icon shortcut"/>
<link href="/s/normalize.css" rel="stylesheet"/>
<link href="/s/styles.css" rel="stylesheet"/>
<!--[if lt IE 8]><link rel="stylesheet" href="/s/ie.css" /><![endif]-->
<link href="/s/print.css" media="print" rel="stylesheet"/>
<link href="/toolbox/javascript/jq

In [140]:
import re
data_raw = cancer_soup.find_all("td")
data_dirty = [i.text.strip() for i in data_raw]
states_dirty = data_dirty[::6]
states_dirty
states = [state.split('\n')[0] for state in states_dirty]
states = states[1:]

In [160]:
data_dirty
incidence_dirty = data_dirty[1::6]
incidence_dirty

def extract_first_number(i):
    match = re.search(r'^\d+(\.\d+)?', i)
    if match:
        return match.group(0)
    else:
        return None

incidence_rate = [extract_first_number(i) for i in incidence_dirty]
incidence_rate = incidence_rate[1:]
incidence_rate = [float(i) if i is not None else None for i in incidence_rate]
incidence_rate

[506.8,
 486.8,
 482.4,
 481.9,
 478.3,
 476.0,
 474.4,
 473.4,
 473.3,
 467.5,
 467.4,
 465.5,
 465.4,
 465.3,
 464.4,
 463.8,
 463.3,
 459.7,
 459.1,
 458.2,
 457.8,
 457.8,
 457.7,
 457.0,
 454.3,
 453.9,
 452.4,
 452.3,
 449.8,
 449.7,
 449.4,
 444.9,
 443.4,
 440.7,
 438.2,
 434.3,
 430.8,
 421.3,
 419.2,
 413.9,
 412.2,
 408.2,
 406.9,
 404.3,
 403.0,
 394.7,
 391.7,
 376.6,
 369.0,
 351.7,
 None,
 None]

In [165]:
data_dirty
rank_dirty = data_dirty[2::6]
rank_dirty

rank = [extract_first_number(i) for i in rank_dirty][1:]
rank = [int(i) if i is not None else None for i in rank]
rank

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 None,
 None,
 None]

In [172]:
avg_annual_dirty = data_dirty[3::6]
avg_annual_dirty = [i.replace(",", "") for i in avg_annual_dirty]
avg_annual_dirty
avg_annual_cases = [int(i) if i != 'data not available' else None for i in avg_annual_dirty][1:]
avg_annual_cases

[27911,
 19197,
 12174,
 53389,
 26491,
 8767,
 114869,
 9293,
 31958,
 17563,
 79630,
 34387,
 16781,
 69011,
 59215,
 54354,
 6329,
 70160,
 10374,
 21269,
 4904,
 5963,
 38378,
 6461,
 3937,
 15648,
 135952,
 3932,
 34586,
 32760,
 38719,
 9122,
 20658,
 27148,
 55927,
 38313,
 28303,
 3124,
 22328,
 7587,
 120668,
 41480,
 2822,
 2891,
 11449,
 174052,
 24801,
 33768,
 9672,
 14350,
 None,
 None]

In [178]:
trend_dirty = data_dirty[4::6]
recent_trend = [None if i == 'data not available' else i for i in trend_dirty][1:]
recent_trend

['stable',
 'rising',
 'stable',
 'falling',
 'rising',
 'falling',
 'falling',
 'stable',
 'rising',
 'rising',
 'falling',
 'stable',
 'stable',
 'rising',
 'falling',
 'stable',
 'falling',
 'stable',
 'rising',
 'falling',
 'stable',
 'falling',
 'falling',
 'falling',
 'stable',
 'falling',
 'falling',
 'stable',
 'falling',
 'stable',
 'stable',
 'stable',
 'stable',
 'falling',
 'falling',
 'falling',
 'falling',
 'stable',
 'falling',
 'stable',
 'stable',
 'stable',
 'falling',
 'falling',
 'falling',
 'stable',
 'stable',
 'falling',
 'stable',
 'stable',
 None,
 None]

In [184]:
fiveyr_dirty = data_dirty[5::6]
fiveyr_dirty

def extract_numerical_value(i):
    match = re.match(r'-?\d+(\.\d+)?', i)
    if match:
        return float(match.group(0))
    else:
        return None

five_year_trend = [extract_numerical_value(i) for i in fiveyr_dirty][1:]
five_year_trend

[-0.3,
 1.0,
 -0.2,
 -0.5,
 0.8,
 -0.6,
 -0.5,
 -0.1,
 0.9,
 1.0,
 -0.7,
 -0.1,
 0.2,
 0.5,
 -0.4,
 -0.1,
 -0.7,
 -0.1,
 0.8,
 -0.6,
 0.1,
 -1.2,
 -0.3,
 -0.4,
 0.2,
 -0.5,
 -0.4,
 -0.3,
 -0.6,
 -0.2,
 -0.4,
 0.4,
 -0.1,
 -0.6,
 -1.1,
 -1.0,
 -1.9,
 0.1,
 -0.7,
 -0.1,
 0.4,
 -0.3,
 -1.4,
 -0.8,
 -0.3,
 -0.6,
 0.2,
 -0.7,
 -0.3,
 0.0,
 None,
 None]

In [191]:
cancer_data_dict = {
    "State": states,
    "Incidence Rate (per 100k)": incidence_rate,
    "Rank": rank,
    "Average Annual Cases": avg_annual_cases,
    "Recent Trend": recent_trend,
    "Five Year Trend": five_year_trend
}

cancer_df = pd.DataFrame(cancer_data_dict)
cancer_df['Region'] = cancer_df['State'].map(us_state_regions_dict)
cancer_df.at[42, 'Region'] = 'northeast'
cancer_df

,State,Incidence Rate (per 100k),Rank,Average Annual Cases,Recent Trend,Five Year Trend,Region
0,Kentucky,506.8,1.0,27911.0,stable,-0.3,southeast
1,Iowa,486.8,2.0,19197.0,rising,1.0,northeast
2,West Virginia,482.4,3.0,12174.0,stable,-0.2,southeast
3,New Jersey,481.9,4.0,53389.0,falling,-0.5,northeast
4,Louisiana,478.3,5.0,26491.0,rising,0.8,southeast
5,New Hampshire,476.0,6.0,8767.0,falling,-0.6,northeast
6,New York,474.4,7.0,114869.0,falling,-0.5,northeast
7,Maine,473.4,8.0,9293.0,stable,-0.1,northeast
8,Minnesota,473.3,9.0,31958.0,rising,0.9,northeast
9,Arkansas,467.5,10.0,17563.0,rising,1.0,southeast
